In [15]:
# google drive에서 파일 가져오기
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# "다음 분기에 어떤 게임을 설계해야 할까"를 도출하는 과정 / 결과 보여주기

dataset read, 어떤 게임을 만들어야 하는지 결론 도출

요구사항.
- 지역에 따라서 선호하는 게임 장르가 다를까 라는 질문에 대답을 하셔야합니다.
- 연도별 게임의 트렌드가 있을까 라는 질문에 대답을 하셔야합니다.
- 출고량이 높은 게임에 대한 분석 및 시각화 프로세스가 포함되어야 합니다.

-----------------

data 이름 설명
+ Name : 게임의 이름입니다.
+ Platform : 게임이 지원되는 플랫폼의 이름입니다.
+ Year : 게임이 출시된 연도입니다.
+ Genre : 게임의 장르입니다.
+ Publisher : 게임을 제작한 회사입니다.
+ NA_Sales : 북미지역에서의 출고량입니다.
+ EU_Sales : 유럽지역에서의 출고량입니다.
+ JP_Sales : 일본지역에서의 출고량입니다.
+ Other_Sales : 기타지역에서의 출고량입니다.


-----------------
부가설명

아래 Sales의 기준은 100만(M)이다.
- 1k = one thousand (1,000)
- 10k = ten thousand (10,000)
- 100k = 100 thousand (100,000)
- 1M = 1000K (100만)

In [16]:
# csv파일 불러오기
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/code_states/vgames2.csv", engine='python')
df.head()

,Unnamed: 0,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,1,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,2,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,4,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,5,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04


In [17]:
df.dtypes

Unnamed: 0       int64
Name            object
Platform        object
Year           float64
Genre           object
Publisher       object
NA_Sales        object
EU_Sales        object
JP_Sales        object
Other_Sales     object
dtype: object

In [18]:
df = df.drop('Unnamed: 0', axis=1)
df.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04


In [19]:
def toFloat(n) :
  try :
    return float(n)
  except ValueError :
    if n[-1] == 'M' :
      return float(n[0:-1])
    elif n[-1] == 'K' :
      return float(n[0:-1])/1000
    else :
      print(n)
  return 'cannot toInt' # 1개라도 있으면 dtype이 object가 됨

def setYear(y) :
  if y > 2021 :
    print(y)
    return 'Na'
  if y > 99 :
    return int(y)
  elif y < 22 :
    return int(2000 + y)
  elif y > 21 :
    return int(1900 + y)

In [20]:
# 계산을 위해 data를 숫자형으로 변환 
df['NA_Sales'] = df['NA_Sales'].apply(toFloat)
df['EU_Sales'] = df['EU_Sales'].apply(toFloat)
df['JP_Sales'] = df['JP_Sales'].apply(toFloat)
df['Other_Sales'] = df['Other_Sales'].apply(toFloat)

df['Year'] = df['Year'].apply(setYear)
# df.head()

In [21]:
df.isnull().sum() # 혹시나 0년 == 2000년일 수도 있으니까 year 변환 뒤 nan을 0으로 치환
df['Year'] = df['Year'].fillna(0)
min(df['Year'])

0.0

In [22]:
# cond = 1979 < df['Year']  df['Year'] < 1985
max(df['NA_Sales'])

41.49

In [23]:
# 연도 등록
def generation(n) :
  return int(n / 100)

df['generation'] = df['Year'].apply(generation)
df.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,generation
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0.00,0.00,0.00,20
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0.00,0.00,0.01,20
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0.00,0.00,0.02,0.00,20
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0.00,0.00,0.00,20
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0.00,0.04,20


 Two-Sample T-test.

 가설검정 예시.
 'Na_Sales(북미지역)'에서의 판매량은 'EU_Sales(유럽지역)'에서의 판매량과 같다(비슷하다)

In [24]:
# scipy를 upgrade해야 아래 코드 실행 가능.
# !pip install --upgrade scipy

In [25]:
from scipy import stats
import numpy as np
game1 = pd.to_numeric(df['NA_Sales'])
game2 = pd.to_numeric(df['EU_Sales'])

print("Na :", np.mean(game1))
print("Na sum:", np.sum(game1))

print("EU :", np.mean(game2))
print("EU sum:", np.sum(game2))
sta, pv = stats.ttest_ind(game1, game2)

print(sta, pv)

Na : 0.26466742981082814
Na sum: 4392.95
EU : 0.14665200626581526
EU sum: 2434.13
15.831371100049903 3.0384925458019944e-56


In [26]:
import matplotlib.pyplot as plt

grouped_df = pd.DataFrame(df.groupby('generation').Genre.count())
grouped_df = grouped_df.reset_index()
grouped_df

,generation,Genre
0,0,271
1,19,1968
2,20,14309


In [27]:
labels = set(df['Genre'])
index = np.arange(len(labels))
labels

{'Action',
 'Adventure',
 'Fighting',
 'Misc',
 'Platform',
 'Puzzle',
 'Racing',
 'Role-Playing',
 'Shooter',
 'Simulation',
 'Sports',
 'Strategy',
 nan}

In [28]:
# plt.bar(index, grouped_df)
# plt.title('Sum of Tips by Day', fontsize=20)

# plt.xlabel('Day', fontsize=18)

# plt.ylabel('Sum of Tips', fontsize=18)

# plt.xticks(index, label, fontsize=15)

# plt.show()

In [29]:
# grouped_df.plot(kind='bar', rot=0)
# plt.title('Dodged Bar Chart by Pandas', fontsize=20)
# plt.legend(fontsize='12')
# plt.show()